In [1]:
# importing required libraries
import matplotlib
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
import numpy as np
import sys
from IPython.display import display
import warnings
import matplotlib.pyplot as plt
import datetime
import pycountry_convert as pc
import pycountry
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim
import math
from sklearn import preprocessing

# library options
%matplotlib inline
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')

In [2]:
def sorting(df, key):
    return df.sort_values(by=key)
def res_index(df):
    return df.reset_index(drop = True)
def copy_df(df):
    return df.copy()
def fillZero(df, values):
    return df.fillna(value=values)
def filterBuyer(df, column):
    return df.loc[df[column] > 0 ]
def filterNoBuyer(df, column):
    return df.loc[df[column] == 0]
def drop_col(df,column):
    return df.drop(labels=column,axis=1,errors = "ignore")
def replace_minus(df, column):
    df[column] = df[column].str.replace('-', " ")
    return df

In [3]:
# load dataset
df1 = pd.read_parquet('../data/final/df1.parquet')
df1_clean = pd.read_parquet('../data/final/df1_clean.parquet')
df_clean = pd.read_parquet('../data/final/df_clean.parquet')
df2 = pd.read_parquet('../data/final/df2.parquet', engine='pyarrow')


In [30]:
df_clean.shape

(3254752, 92)

In [31]:
df_clean

,n_ad_reward_claims,n_ad_reward_fails,n_ads_watched,n_battlepass_lvls_finished,masked_feature_0,masked_feature_1,masked_feature_2,masked_feature_3,masked_feature_4,masked_feature_5,masked_feature_6,masked_feature_7,masked_feature_8,masked_feature_13,masked_feature_14,masked_feature_15,masked_feature_20,masked_feature_21,masked_feature_22,masked_feature_23,masked_feature_24,masked_feature_25,n_instant_awards_claims,masked_feature_27,masked_feature_28,masked_feature_29,n_extra_challenge_buys,connected_fb,connect_fb_attempt,n_friend_installs,masked_feature_30,masked_feature_31,n_accept_all_friends,n_accept_friend,n_clicked_add_friends,n_friend_searches,masked_feature_32,masked_feature_35,masked_feature_36,masked_feature_37,masked_feature_38,n_levels_completed,device_language,n_missions_completed,first_time_zone,first_network_type,first_login_country,first_device_manufacturer,n_package_info_offers_viewed,n_payments_package_key,sum_payments_package_key,n_package_tips_offers_viewed,rated_game,n_remove_ad_clicks,masked_feature_39,n_sessions_ended,total_session_duration,max_session_end_player_level,n_sessions_started,min_session_start_player_level,n_time_skip_buys,n_viplevels_completed,bigmac_dollar_price,gdp_per_capita,ram_max,form_factor,screen_density,avg_country_spend_30d,avg_country_spend_30d_first3d,screen_width,screen_height,most_frequent_network_type,nunique_network_types,masked_feature_41,nunique_iaps_bought,first_login_weekday,battlepass_0.0,battlepass_22.0,battlepass_23.0,battlepass_8008.0,first_login_day_time,first_login_country_is_mfreq,time_between_last_purchase_last_login,sum_payments_package_key_ltv,n_payments_package_key_ltv,continent,sum_payments_package_key_ltv_uncapped,sum_day_1,sum_day_2,sum_day_3,clumpiness,gradient
account_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100000293,0,0,0,2,31,22.0,22,9.0,9,12,9,0,309,5.0,24.0,0.0,210.0,20.0,5.0,24.0,0.0,220.0,0,17,24,1,0,0,1,0,0,0,0,0,0,0,0,11,220.0,36,0,0,en,87,+05:30,WIFI,IN,Xiaomi,205,0,0.00,17,0,0,32,31,10301955.0,7,31,0,0,0,2.588909,1900.706810,4900.906098,Phone,423.446727,16.751835,8.488506,1026.497824,2204.589554,WIFI,2.0,0.0,0.0,Saturday,False,False,False,False,morning,1,1.000000,0.00,0,AS,0.00,0.00,0.0,0.0,0.000000,0.0
100000333,0,0,5,2,31,20.0,20,11.0,11,17,11,0,307,11.0,24.0,0.0,200.0,0.0,10.0,21.0,0.0,50.0,0,5,24,0,0,0,0,0,0,0,0,0,0,0,4,4,20.0,36,1,0,ru,91,+03:00,WIFI,RU,Xiaomi,51,0,0.00,19,0,0,34,13,8417646.0,7,14,0,0,0,1.808924,10126.721794,3613.000000,Phone,320.000000,23.861452,5.945218,720.000000,1560.000000,WIFI,1.0,0.0,0.0,Saturday,False,False,False,False,morning,1,1.000000,0.00,0,EU,0.00,0.00,0.0,0.0,0.000000,0.0
100000463,0,0,0,0,2,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,36,0,0,en,5,+07:00,WIFI,AS_Others,samsung,0,0,0.00,0,0,0,33,1,44970.0,1,2,0,0,0,2.643337,46323.863442,5576.000000,Phone,361.753574,57.015214,19.000000,1080.000000,2009.000000,NULL,2.0,0.0,0.0,Saturday,False,False,False,False,morning,1,1.000000,0.00,0,AS,0.00,0.00,0.0,0.0,0.000000,0.0
100000624,0,0,0,0,6,0.0,0,0.0,0,2,0,0,0,0.0,3.0,0.0,30.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,36,0,0,de,19,+01:00,WIFI,EU_Others,HUAWEI,0,0,0.00,0,0,0,1,4,2525386.0,1,4,0,0,0,3.669850,48105.364625,5975.634571,Phone,444.880795,41.824738,16.015051,1042.137475,2195.235937,WIFI,1.0,0.0,0.0,Saturday,False,False,False,False,eve,1,1.000000,0.00,0,EU,0.00,0.00,0.0,0.0,0.000000,0.0
100000713,0,0,0,0,4,0.0,0,0.0,0,1,0,0,0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,36,0,0,en,10,+01:00,WIFI,EU_Others,samsung,0,0,0.00,0,0,0,33,2,669772.0,1,2,0,0,0,3.669850,44594.378108,3713.000000,Tablet,240.000000,47.944754,16.144416,1080.000000,2000.000000,WIFI,1.0,0.0,0.0,Saturday,False,False,False,False,morning,1,1.000000,0.00,0,EU,0.00,0.00,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [4]:
x_0 = df_clean[(df_clean["sum_payments_package_key_ltv_uncapped"] <= 2.99) & (df_clean["sum_payments_package_key_ltv_uncapped"] != 0)].shape[0]
x_1 = df_clean[(df_clean["sum_payments_package_key_ltv_uncapped"] <= 14.95) & (df_clean["sum_payments_package_key_ltv_uncapped"] > 2.99)].shape[0]
x_2 = df_clean[(df_clean["sum_payments_package_key_ltv_uncapped"] > 14.95)].shape[0]

In [5]:
print("Number of accounts: " + str(df_clean.shape[0]))
print("Number of non buyers: " + str(df_clean[df_clean["sum_payments_package_key_ltv_uncapped"] == 0].shape[0]))
print("Total number of buyers: " + str(df_clean[df_clean["sum_payments_package_key_ltv_uncapped"] != 0].shape[0]))
print("Number of buyers after day 3: " + str(df_clean[df_clean["sum_payments_package_key"] != 0].shape[0]))
print("Number of buyers that spent below 3: " + str(x_0))
print("Number of buyers that spent between 3 and 14.95: " + str(x_1))
print("Number of buyers that spent above 14.95: " + str(x_2))

Number of accounts: 3254752
Number of non buyers: 3067174
Total number of buyers: 187578
Number of buyers after day 3: 139739
Number of buyers that spent below 3: 67785
Number of buyers that spent between 3 and 14.95: 57506
Number of buyers that spent above 14.95: 62287


# Profile Reports

In [5]:
df1_full_profile_minimal = ProfileReport(df1, title="DF1 Full Profiling Report Minimal", minimal=True)
df1_full_profile_minimal.to_file("../data/profiling/df1_full_profile_report_minimal.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 71.36it/s]


In [8]:
df1_buyers = df1[df1['sum_payments_package_key_ltv'] > 0]
df1_buyers_full_profile_minimal = ProfileReport(df1_buyers, title="DF1 Buyers Profiling Report Minimal", minimal=True)
df1_buyers_full_profile_minimal.to_file("../data/profiling/df1_buyers_profile_report_minimal.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 83.26it/s]


In [11]:
df1_clean_full_profile_minimal = ProfileReport(df1_clean, title="DF1 Clean Full Profiling Report Minimal", minimal=True)
df1_clean_full_profile_minimal.to_file("../data/profiling/df1_clean_full_profile_report_minimal.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 99.89it/s]


In [12]:
df1_clean_buyers = df1_clean[df1_clean['sum_payments_package_key_ltv'] > 0]
df1_clean_buyers_full_profile_minimal = ProfileReport(df1_clean_buyers, title="DF1 Clean Buyers Profiling Report Minimal", minimal=True)
df1_clean_buyers_full_profile_minimal.to_file("../data/profiling/df1_clean_buyers_profile_report_minimal.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 99.91it/s]


In [16]:
df_clean_full_profile_minimal = ProfileReport(df_clean, title="DF Clean Full Profiling Report Minimal", minimal=True)
df_clean_full_profile_minimal.to_file("../data/profiling/df_clean_full_profile_report_minimal.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 90.83it/s]
